In [1]:
import os
import dotenv
import numpy as np
from dataclasses import replace
import matplotlib.pyplot as plt
from pulser import Pulse, Register, Sequence
from pulser.register.register_layout import RegisterLayout
from pulser.devices import Chadoq2
from pulser.channels.dmm import DMM
from pulser.waveforms import InterpolatedWaveform, ConstantWaveform
from pasqal_cloud import SDK
from pasqal_cloud.device import EmulatorType

In [2]:
N = 8
L = 7.0
omega_max = 15

trap_coordinates = [(i*L, 0.0) for i in range(N)]
weights = np.ones((N))

register = Register.from_coordinates(trap_coordinates, center=False, prefix="q")
# register.draw(
#     blockade_radius=Chadoq2.rydberg_blockade_radius(omega_max),
#     draw_graph=False,
#     draw_half_radius=True,
# )

In [3]:
weights = np.array([0.325, 0] + list(np.linspace(0.5, 1, N-2)))
weights /= weights.sum()

dmm = DMM(
    clock_period=4,
    min_duration=16,
    max_duration=2**26,
    mod_bandwidth=8,
    bottom_detuning=-800,  # detuning between 0 and -20 rad/µs
)
# Chadoq3 = replace(
#     Chadoq2.to_virtual(),
#     dmm_objects=(dmm, DMM()),
#     reusable_channels=True,
#     max_radial_distance=100,
#     rydberg_level = 70,
#     max_atom_num = 100
# )

register_layout = RegisterLayout(trap_coordinates)
detuning_map = register_layout.define_detuning_map(
    {i: weights[i] for i in range(N)}  # mapping between trap ids and weights
)
# detuning_map.draw(labels=range(N))

In [9]:
delta_abs = 8
T = 10000 # in ns, evolution time

seq = Sequence(register, Chadoq2)

seq.config_detuning_map(detuning_map, "dmm_0")
seq.declare_channel("ch0", "rydberg_global")

adiabatic_pulse = Pulse(
    amplitude = InterpolatedWaveform(T, [1e-9, omega_max, 1e-9]), # By default, interpolate just connects the dots with cubic functions
    detuning = InterpolatedWaveform(T, [-delta_abs, delta_abs]),
    phase = 0
)

seq.add(adiabatic_pulse, "ch0")
seq.add_dmm_detuning(ConstantWaveform(T, -delta_abs / 2 / weights.max()), "dmm_0")
# seq.draw(
#         mode="input+output",
#         draw_qubit_det=True,
#          draw_qubit_amp=True)

In [10]:
serialized_sequence = seq.to_abstract_repr()

In [11]:
dotenv.load_dotenv()
project_id = os.environ.get('PASQAL_PROJECT_ID')
username = os.environ.get('PASQAL_USERNAME')
password = os.environ.get('PASQAL_PASSWORD')

if not all([project_id, username, password]):
    raise ValueError("Missing PASQAL credentials. Please set environment variables.")

In [12]:
sdk = SDK(username=username, password=password, project_id=project_id)

job = {"runs": 50, "variables": {}}

batch = sdk.create_batch(
    serialized_sequence, [job], emulator=EmulatorType.EMU_TN
)

for job in batch.ordered_jobs:
    print(f"job-id: {job.id}, status: {job.status}, result: {job.result}")


job-id: c9743738-bad2-4844-b942-5d034284552b, status: PENDING, result: None


In [26]:
print(job.status)

PENDING


In [15]:
result = sdk.get_batch(batch.id)
print(result.ordered_jobs[0].result)
count_dict = result.ordered_jobs[0].result


None


In [14]:
count_dict = dict(sorted(count_dict.items(), key=lambda item: item[1], reverse=True))
plt.figure()
plt.xlabel("bitstrings")
plt.ylabel("counts")
plt.bar(count_dict.keys(), count_dict.values(), width=0.5)
plt.xticks(rotation="horizontal")
plt.show()

AttributeError: 'NoneType' object has no attribute 'items'